# Contrôler Docker depuis un container (Jupyter) #

## Installation de docker dans le container ##

Des droits root, sont nécessaires pour installer docker. Depuis l'hôte, exécuter la commande suivante :

```bash
docker exec -it -u 0 jupyter sh -c "echo 'jovyan ALL=(ALL) NOPASSWD:ALL' >> /etc/sudoers"
```

NOTES :
- Ici, `jovyan` est l'utilisateur par défaut, de l'image officielle Jupyter.
- Alternativement, le conteneur peut être démarré avec la variable d'environnement `GRANT_SUDO=yes`.

Procéder à l'installation

In [ ]:
%%bash
curl -fsSL https://get.docker.com |sudo sh -
sudo usermod -aG docker jovyan

## Connection au docker de l'hôte ##

### Via les volumes ###

Le socket docker de l'hôte est monté dans le conteneur.
Néanmoins, pour des raisons de droits il faut utiliser docker avec `sudo`

In [ ]:
! sudo docker ps

### Via TCP ###

La configuration permet de ne utiliser sudo avec docker

#### Configuration de l'hôte sous Windows ou MacOS (avec Docker Desktop):  ####
Aller dans *Setting > General* et cocher *Expose daemon on tcp:...*        

#### Configuration de l'hôte sous Linux ####

Il faut démarré le démon avec l'option `-H tcp://127.0.0.1:2375`

#### Configuration du conteneur ####

##### Accès depuis le noteook #####

Exécuter la ligne suivante pour dans chaque notebook où l'on accède à docker :

In [ ]:
%env DOCKER_HOST=tcp://host.docker.internal:2375

NOTE: Sous linux, remplacer `host.docker.internal` par l'ip de l'hôte.

In [ ]:
! docker ps

##### Accès depuis un terminal #####

In [ ]:
%%bash
IP_HOST="host.docker.internal"
echo "export DOCKER_HOST=\"tcp://host.docker.internal:2375\"" >> ~/.bashrc && source ~/.bashrc

## Installation de docker-compose ##

In [ ]:
compose_ver=$(curl --silent "https://api.github.com/repos/docker/compose/releases/latest" | grep tag_name | sed "s/ *\"tag_name\": *\"\\(.*\\)\",*/\\1/")
sudo curl -L "https://github.com/docker/compose/releases/download/$compose_ver/docker-compose-$(uname -s)-$(uname -m)" -o /usr/local/bin/docker-compose
sudo chmod +x /usr/local/bin/docker-compose
docker-compose -v

##### Suppression de l'utilisateur 'jovyan' des sudoers ##

Cette étape est optionelle (à proscrire, si on se connecte via le socket docker). Elle ne concerne que l'ajout via la commande `docker exec` préconisée plus haut

In [ ]:
! sudo head -n -1 /etc/sudoers > tmp;sudo mv tmp /etc/sudoers